# Introducing Hugging Face's Smolagents

A stand-alone notebook, basicly a stripped-down copy of the article [Hugging Face's Smolagents: A Guide With Examples](https://app.datacamp.com/learn/tutorials/smolagents) in Datacamp. 

## What and why it is

As described by Hugging Face’s [announcement blog](https://huggingface.co/blog/smolagents), **smolagents** is *“a very simple library that unlocks agentic capabilities for language models.”* 

At their heart, agents are powered by LLMs to dynamically solve a task by observing their environments, making plans, and executing those plans given their toolbox. Building these agents, while not impossible, requires you to write from scratch many components. These components ensure that the agents function properly without burning through your API credit and execution time. Agentic frameworks make this easier so you don’t have to reinvent the wheels.

AI agent frameworks are often criticized with two points:

- They build too many layers of abstraction, making them rigid and challenging to debug.
- They focus on “workflows” rather than building agents that can dynamically collaborate on their own.

On the other hand, smolagents has (we are told) qualities that make it very promising for these agentic applications:

- The framework’s abstractions are kept at a minimum.
- While most frameworks have the agents define their actions in JSON/text format, smolagents’ main approach is **Code Agents** in which actions are written as Python code snippets (this is different from agents that write code).
- Being a Hugging Face framework, smolagents integrates well with the Hub and the Transformers library. You can use many models from the hub (some of them you can only use as a Pro user), and you can also work with proprietary models from OpenAI, Anthropic, etc.
- You can easily utilize the already-provided tools, or define your custom tools with minimum effort, almost as simple as writing a Python function.

## Building a Demo Project With Smolagents

In this section, we will build a simple demo with smolagents. Our application will have an agent get the most upvoted paper on the Hugging Face [Daily Papers page](https://huggingface.co/papers). We build our custom tools for the agent and see it work in action.


### Setup

In [ ]:
!pip install -qU smolagents

### Building custom tools

While the framework provides built-in tools (e.g. DuckDuckGoSearchTool), building your custom tools is just as straightforward. We'll build four tools, each for a particular purpose:

- Getting the title of the top daily paper.
- Getting the ID of the paper using its title.
- Downloading the paper from arXiv with the ID.
- Reading the downloaded PDF file.

It’s important to ensure agents clearly understand which tool to use and how to use it. To achieve this, be as explicit as possible when defining these tools:

- Choose an informative name for the function.
- The inputs and outputs of the function should have type hints.
- A description of the tool's purpose must be included. This serves as a manual to the agent.

#### Tool1: Get the title of the top daily paper

In [ ]:
from smolagents import tool 
# import packages that are used in our tools
import requests
from bs4 import BeautifulSoup
import json

@tool
def get_hugging_face_top_daily_paper() -> str:
    """
    This is a tool that returns the most upvoted paper on Hugging Face daily papers.
    It returns the title of the paper
    """
    try:
      url = "<https://huggingface.co/papers>"
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      soup = BeautifulSoup(response.content, "html.parser")

      # Extract the title element from the JSON-like data in the "data-props" attribute
      containers = soup.find_all('div', class_='SVELTE_HYDRATER contents')
      top_paper = ""

      for container in containers:
          data_props = container.get('data-props', '')
          if data_props:
              try:
                  # Parse the JSON-like string
                  json_data = json.loads(data_props.replace('&quot;', '"'))
                  if 'dailyPapers' in json_data:
                      top_paper = json_data['dailyPapers'][0]['title']
              except json.JSONDecodeError:
                  continue

      return top_paper
    except requests.exceptions.RequestException as e:
      print(f"Error occurred while fetching the HTML: {e}")
      return None

#### Tool 2: Get the paper ID by its title

In [ ]:
from huggingface_hub import HfApi

@tool
def get_paper_id_by_title(title: str) -> str:
    """
    This is a tool that returns the arxiv paper id by its title.
    It returns the title of the paper

    Args:
        title: The paper title for which to get the id.
    """
    api = HfApi()
    papers = api.list_papers(query=title)
    if papers:
        paper = next(iter(papers))
        return paper.id
    else:
        return None

#### Tool 3: Download the paper from arXiv with the ID

In [ ]:
!pip install -qU arxiv

In [ ]:
import arxiv

@tool
def download_paper_by_id(paper_id: str) -> None:
    """
    This tool gets the id of a paper and downloads it from arxiv. It saves the paper locally 
    in the current directory as "paper.pdf".

    Args:
        paper_id: The id of the paper to download.
    """
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
    paper.download_pdf(filename="paper.pdf")
    return None

#### Tool 4: Read a pdf file

In [ ]:
!pip install -qU pypdf

In [ ]:
from pypdf import PdfReader

@tool
def read_pdf_file(file_path: str) -> str:
    """
    This function reads the first three pages of a PDF file and returns its content as a string.
    Args:
        file_path: The path to the PDF file.
        pages_to_read:
    Returns:
        A string containing the content of the PDF file.
    """
    content = ""
    reader = PdfReader('paper.pdf')
    print(len(reader.pages))
    pages = reader.pages[:3]
    for page in pages:
        content += page.extract_text()
    return content

## A sidebar (not in the original paper): Qwen models

Qwen is  a series of large language models (LLMs) and large multimodal models (LMMs) that are designed to perform various tasks, including natural language understanding, mathematical problem-solving, and coding.

Some of the key features and capabilities of Qwen include:

- Strong performance: Qwen models have demonstrated competitive performance on various benchmarks, often outperforming similar-sized models.
- Multimodal capabilities: Some Qwen models are multimodal, meaning they can process and understand both text and images.
- Open source: Alibaba Cloud has made some of the Qwen models available open source, encouraging further research and development.

Resources:
- About Qwen: https://qwenlm.github.io/about/
- GitHub: This is where you'll find the code, documentation, and updates for the Qwen models: https://github.com/QwenLM/Qwen
- Hugging Face: You can explore the Qwen models, download them, and even try them out in your browser: https://huggingface.co/QwenLM



## Running the Agent
With our tools set up, we can now initialize and run our agent. We use the Qwen2.5-Coder-32B-Instruct model, which is free to use. The tools an agent needs can be passed while defining the agent. You can see the process of defining and running an agent requires minimum code.

As the agent operates, it outputs its process step by step. This allows us to see how it defines its actions in code while utilizing the custom tools we’ve provided.

Before running the agent, we must set the API key. We're using GitHub Secrets for storing the keys.

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("HF_API_KEY")

In [16]:
from smolagents import CodeAgent, HfApiModel

model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"
#model_id = "Qwen/Qwen2.5-Coder-14B-Instruct"

#model_id = "meta-llama/CodeLlama-7b-Instruct-hf"

model = HfApiModel(model_id=model_id, token=os.environ["HF_API_KEY"])
agent = CodeAgent(tools=[get_hugging_face_top_daily_paper,
                         get_paper_id_by_title,
                         download_paper_by_id,
                         read_pdf_file],
                  model=model,
                  add_base_tools=True)

agent.run(
    "Summarize today's top paper on Hugging Face daily papers by reading it.",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Summarize today's top paper on Hugging Face daily papers by reading it.                                         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
(ReadTimeoutError("HTTPSConnectionPool(host='api-inference.huggingface.co', port=443): Read timed out. (read 
timeout=120)"), '(Request ID: 1373c34d-f362-4181-9c2f-947f850b84b8)')

[Step 0: Duration 120.08 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Self-Discover: Large Language Models Self-Compose Reasoning Structures introduces a framework for  
  LLMs to self-discover and utilize task-specific reasoning structures, enhancing performance on complex           
  reasoning tasks compared to traditional prompting methods.")                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Self-Discover: Large Language Models Self-Compose Reasoning Structures introduces a framework 
for LLMs to self-discover and utilize task-specific reasoning structures, enhancing performance on complex 
reasoning tasks compared to traditional prompting methods.

[Step 5: Duration 32.43 seconds| Input tokens: 33,418 | Output tokens: 1,485]

'Self-Discover: Large Language Models Self-Compose Reasoning Structures introduces a framework for LLMs to self-discover and utilize task-specific reasoning structures, enhancing performance on complex reasoning tasks compared to traditional prompting methods.'

## Conclusion and further resources

Well that wasn't a complete success...

### Further reading:

- [Smolagents documentation](https://huggingface.co/docs/smolagents/index)
- [Smolagents GitHub repo](https://github.com/huggingface/smolagents)
- [About Gwen2.5-Coder Series](https://qwenlm.github.io/blog/qwen2.5-coder-family/)